In [6]:
# %pip install opencv-python numpy matplotlib

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_images(images, titles, figsize=(15, 5)):
    """Helper function to display multiple images side by side"""
    fig, axes = plt.subplots(1, len(images), figsize=figsize)
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if len(images) == 1:
            ax = axes
        else:
            ax = axes[i]
        ax.imshow(img, cmap='gray' if len(img.shape) == 2 else None)
        ax.set_title(title)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def denoise_image(image):
    """Apply denoising using Non-local Means Denoising"""
    return cv2.fastNlMeansDenoising(image) if len(image.shape) == 2 else cv2.fastNlMeansDenoisingColored(image)

def apply_morphology(image, operation='erosion', kernel_size=5):
    """Apply morphological operations (erosion or dilation)"""
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    if operation == 'erosion':
        return cv2.erode(image, kernel, iterations=1)
    else:  # dilation
        return cv2.dilate(image, kernel, iterations=1)

def apply_threshold(image, method='binary'):
    """Apply different thresholding methods"""
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if method == 'binary':
        _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    elif method == 'adaptive':
        thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
    elif method == 'otsu':
        _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def detect_edges(image, method='canny'):
    """Apply edge detection"""
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if method == 'canny':
        return cv2.Canny(image, 100, 200)
    elif method == 'sobel':
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
        return cv2.magnitude(sobelx, sobely)

def equalize_histogram(image):
    """Apply histogram equalization"""
    if len(image.shape) == 2:
        return cv2.equalizeHist(image)
    else:
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        # Apply histogram equalization to L channel
        l_eq = cv2.equalizeHist(l)
        # Merge channels and convert back to BGR
        lab_eq = cv2.merge([l_eq, a, b])
        return cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)



# Example usage:
image_path = 'low quality documents/1.png'
img = cv2.imread(image_path)
if img is None:
    raise ValueError(f"Could not read image at {image_path}")

# Convert BGR to RGB for display
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Apply different techniques
denoised = cv2.cvtColor(denoise_image(img), cv2.COLOR_BGR2RGB)
eroded = cv2.cvtColor(apply_morphology(img, 'erosion'), cv2.COLOR_BGR2RGB)
dilated = cv2.cvtColor(apply_morphology(img, 'dilation'), cv2.COLOR_BGR2RGB)
threshold = apply_threshold(img, 'otsu')
edges = detect_edges(img)
equalized = cv2.cvtColor(equalize_histogram(img), cv2.COLOR_BGR2RGB)
